In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tqdm
from scipy.optimize import curve_fit
import h5py
import pyvisa
import time
from scipy.signal import savgol_filter
import scipy.signal as signal
from scipy import interpolate
from matplotlib.ticker import MaxNLocator
from scipy.optimize import least_squares
import nidaqmx
import pandas as pd

# If the imports don't work, press F1, find "Developer: Reload Window"

factor = 2
SMALL_SIZE = factor*8
MEDIUM_SIZE = factor*10
BIGGER_SIZE = factor*12
plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title4

rm = pyvisa.ResourceManager()
rm.list_resources()


('ASRL1::INSTR', 'ASRL9::INSTR')

In [38]:
import os

os.chdir("C:\\Users\\CeNTREX\\Documents\\1-3-IR-high_voltage\\Emma leakage current\\2-27-2025_test")

In [2]:
device_name = "dev2"
DAQ_read_channels = ["ai1", "ai2"]
DAQ_write_channel = "ao0"
DAQ_timeout = 5000

In [3]:
#scope = rm.open_resource('USB0::0x1AB1::0x04CE::DS1ZA204519116::INSTR')
#scope.query("*IDN?")

### Data collection functions

In [7]:
# RAMP TO VOLTAGE 


def ramp_to(start_v, end_v, total_ramp_time=60, v_step=0.001):
    """
    Ramps the voltage on the DAQ output from start_v to end_v over total_ramp_time, using steps of v_step.
    
    Parameters:
        start_v (float): Starting voltage, must be between 0 and 10.1.
        end_v (float): Ending voltage, must be between 0 and 10.1.
        total_ramp_time (float): Total time for the voltage ramp (default is 60 seconds).
        v_step (float): Voltage step (default is 0.001 volts).
    """
    # Check if input voltages are within the valid range
    if not (0 <= start_v <= 10.1) or not (0 <= end_v <= 10.1):
        print("Error: Voltage must be between 0 and 10.1 volts.")
        return
    
    # Calculate the total number of steps and the time interval (dt) between steps
    voltage_range = abs(end_v - start_v)
    num_steps = int(voltage_range / v_step)  # Number of voltage steps
    dt = total_ramp_time / num_steps  # Time between each voltage step
    print(dt)
    
    # Set the initial voltage to start_v
    current_voltage = start_v

    try:
        with nidaqmx.Task() as task:
            # Configure the AO0 channel on device "dev2"
            task.ao_channels.add_ao_voltage_chan("dev2/ao0", min_val=0.0, max_val=10.1)
            
            # Set the initial voltage to start_v
            task.write(current_voltage)
            print(f"Starting ramp from {start_v}V to {end_v}V with step {v_step}V.")
            
            # Determine the direction of the ramp (up or down)
            step_direction = 1 if end_v > start_v else -1
            
            for _ in range(num_steps):
                # Update the current voltage in the direction of the ramp
                current_voltage += step_direction * v_step
                
                # Output the new voltage to the DAQ
                task.write(current_voltage)
                print(f"current voltage: {current_voltage}V.")
                
                # Wait for the calculated time step
                time.sleep(dt)
                
                # Placeholder for additional error checking or exception handling
                # You can add custom error handling code here
                pass

    except Exception as e:
        # If any exception occurs, handle it here
        print(f"An exception occurred: {str(e)}. Exiting ramp.")
        return

    print(f"Successfully ramped from {start_v}V to {current_voltage}V.")


# Example usage:
# ramp_to(0.5, 5.0)  # Ramps from 0.5V to 5.0V over 60 seconds with a step size of 0.001V




# RAMP TO BUT SAVE DAQ AND MULTIMETER DATA

def ramp_and_record(start_v, end_v, total_ramp_time=60, v_step=0.001, filename="ramp_and_record"):
    """
    Ramps the voltage on the DAQ output from start_v to end_v over total_ramp_time, using steps of v_step.

    Parameters:
        start_v (float): Starting voltage, must be between 0 and 10.1.
        end_v (float): Ending voltage, must be between 0 and 10.1.
        total_ramp_time (float): Total time for the voltage ramp (default is 60 seconds).
        v_step (float): Voltage step (default is 0.001 volts).
    """
    # Check if input voltages are within the valid range
    if not (0 <= start_v <= 10.1) or not (0 <= end_v <= 10.1):
        print("Error: Voltage must be between 0 and 10.1 volts.")
        return

    # Calculate the total number of steps and the time interval (dt) between steps
    voltage_range = abs(end_v - start_v)
    num_steps = int(np.ceil(voltage_range / v_step))  # Number of voltage steps
    dt = total_ramp_time / num_steps  # Time between each voltage step

    # Set the initial voltage to start_v
    current_voltage = start_v

    # DAQ settings
    sample_rate = 10  # Samples per second
    num_samples = int((sample_rate * (total_ramp_time * 1.1)*((num_steps + 1)/(num_steps))))

    timestamp = time.strftime('%Y-%m-%d_%H-%M-%S')

    with nidaqmx.Task() as read_task:
        for ch in DAQ_read_channels:
            read_task.ai_channels.add_ai_voltage_chan(f"{device_name}/{ch}")

        read_task.timing.cfg_samp_clk_timing(sample_rate, samps_per_chan=num_samples)

        # Data logging for ramp started
        #time.sleep(0.1)

        # Start the DAQ task for data logging
        read_task.start()


        try:
            with nidaqmx.Task() as task:
                # Configure the AO0 channel on device "dev2"
                task.ao_channels.add_ao_voltage_chan("dev2/ao0", min_val=0.0, max_val=10.1)

                # Set the initial voltage to start_v
                task.write(current_voltage)
                print(f"Starting ramp from {start_v}V to {end_v}V with step {v_step}V.")

                # Determine the direction of the ramp (up or down)
                step_direction = 1 if end_v > start_v else -1

                for _ in range(num_steps):

                    temp_timer = time.time()

                    #Wait for the calculated time step
                    time.sleep(dt)

                    # Update the current voltage in the direction of the ramp
                    current_voltage += step_direction * v_step

                    # Output the new voltage to the DAQ
                    task.write(current_voltage)
                    print(f"current voltage: {current_voltage}V.")

                    # Placeholder for additional error checking or exception handling
                    pass

        except Exception as e:
            # If any exception occurs, handle it here
            print(f"An exception occurred: {str(e)}. Exiting ramp.")
            return
        
        temp_timer = time.time()

        print(f"Successfully ramped from {start_v}V to {current_voltage}V.")

        # Read the data with the specified timeout
        DAQ_data = read_task.read(number_of_samples_per_channel=num_samples, timeout=DAQ_timeout)
        elapsed_time = [i / sample_rate for i in range(num_samples)]

    # Convert to a pandas DataFrame
    df = pd.DataFrame({
        'DAQ_Elapsed_Time': elapsed_time,
        'V_mon': DAQ_data[0],
        'I_mon': DAQ_data[1]
    })

    # Save to CSV for multimeter data
    full_file_name = f"{filename}_{start_v:.3f}V_to_{end_v:.3f}V_step_{v_step}V_in_{total_ramp_time}s_{timestamp}_DAQ.csv"
    df.to_csv(full_file_name, index=False)

# Example usage:
# ramp_and_record(0.1, 0, 30, 0.001)  # Ramps from 0.1V to 0V over 30 seconds with a step size of 0.001V


def record(total_ramp_time=60, filename="record"):

    # DMM settings
    t0=time.time()

    # DAQ settings
    sample_rate = 10  # Samples per second
    num_samples = int((sample_rate * (total_ramp_time * 1.1)))
    timestamp = time.strftime('%Y-%m-%d_%H-%M-%S')
  

    with nidaqmx.Task() as read_task:
        for ch in DAQ_read_channels:
            read_task.ai_channels.add_ai_voltage_chan(f"{device_name}/{ch}")

        read_task.timing.cfg_samp_clk_timing(sample_rate, samps_per_chan=num_samples)

        # Start the DAQ task for data logging
        read_task.start()
        

        print(f"Successfully recorded.")

        # Read the data with the specified timeout
        DAQ_data = read_task.read(number_of_samples_per_channel=num_samples, timeout=DAQ_timeout)
        elapsed_time = [i / sample_rate for i in range(num_samples)]

    # Convert to a pandas DataFrame
    df = pd.DataFrame({
        'DAQ_Elapsed_Time': elapsed_time,
        'V_mon': DAQ_data[0],
        'I_mon': DAQ_data[1]
    })

    # Save to CSV for multimeter data
    full_file_name = f"{filename}_recording_{total_ramp_time}s_{timestamp}_DAQ.csv"   
    df.to_csv(full_file_name, index=False)

    


In [42]:
ramp_and_record(0.,10.,120,0.02)


Starting ramp from 0.0V to 10.0V with step 0.02V.
current voltage: 0.02V.
current voltage: 0.04V.
current voltage: 0.06V.
current voltage: 0.08V.
current voltage: 0.1V.
current voltage: 0.12000000000000001V.
current voltage: 0.14V.
current voltage: 0.16V.
current voltage: 0.18V.
current voltage: 0.19999999999999998V.
current voltage: 0.21999999999999997V.
current voltage: 0.23999999999999996V.
current voltage: 0.25999999999999995V.
current voltage: 0.27999999999999997V.
current voltage: 0.3V.
current voltage: 0.32V.
current voltage: 0.34V.
current voltage: 0.36000000000000004V.
current voltage: 0.38000000000000006V.
current voltage: 0.4000000000000001V.
current voltage: 0.4200000000000001V.
current voltage: 0.4400000000000001V.
current voltage: 0.46000000000000013V.
current voltage: 0.48000000000000015V.
current voltage: 0.5000000000000001V.
current voltage: 0.5200000000000001V.
current voltage: 0.5400000000000001V.
current voltage: 0.5600000000000002V.
current voltage: 0.5800000000000

In [44]:
ramp_and_record(10.,0.,120,0.02)

Starting ramp from 10.0V to 0.0V with step 0.02V.
current voltage: 9.98V.
current voltage: 9.96V.
current voltage: 9.940000000000001V.
current voltage: 9.920000000000002V.
current voltage: 9.900000000000002V.
current voltage: 9.880000000000003V.
current voltage: 9.860000000000003V.
current voltage: 9.840000000000003V.
current voltage: 9.820000000000004V.
current voltage: 9.800000000000004V.
current voltage: 9.780000000000005V.
current voltage: 9.760000000000005V.
current voltage: 9.740000000000006V.
current voltage: 9.720000000000006V.
current voltage: 9.700000000000006V.
current voltage: 9.680000000000007V.
current voltage: 9.660000000000007V.
current voltage: 9.640000000000008V.
current voltage: 9.620000000000008V.
current voltage: 9.600000000000009V.
current voltage: 9.580000000000009V.
current voltage: 9.56000000000001V.
current voltage: 9.54000000000001V.
current voltage: 9.52000000000001V.
current voltage: 9.50000000000001V.
current voltage: 9.480000000000011V.
current voltage: 9

In [29]:
ramp_to(0.1,0.,30,0.01)

3.0
Starting ramp from 0.1V to 0.0V with step 0.01V.
current voltage: 0.09000000000000001V.
current voltage: 0.08000000000000002V.
current voltage: 0.07000000000000002V.
current voltage: 0.06000000000000002V.
current voltage: 0.05000000000000002V.
current voltage: 0.040000000000000015V.
current voltage: 0.030000000000000013V.
current voltage: 0.02000000000000001V.
current voltage: 0.01000000000000001V.
current voltage: 1.0408340855860843e-17V.
Successfully ramped from 0.1V to 1.0408340855860843e-17V.


In [43]:
record(120,"rec_at_10")

Successfully recorded.


In [7]:

#for i in [0,1,2,3,4,5,6,7,8,9]:
 #   ramp_and_record(i,i+0.9,50,0.01)
  #  ramp_and_record(i+0.9, i+1, 20, 0.1)

record(6,"test")

Successfully recorded.


In [8]:
for ii in [10,9,8,7,6,5,4,3,2,1]:
    ramp_and_record(ii, ii - 0.1, 100, 0.1)
    ramp_and_record(ii - 0.1, ii - 1,50,0.01)

Starting ramp from 10V to 9.9V with step 0.1V.
Successfully ramped from 10V to 9.9V.
Starting ramp from 9.9V to 9V with step 0.01V.
Successfully ramped from 9.9V to 8.99000000000002V.
Starting ramp from 9V to 8.9V with step 0.1V.
Successfully ramped from 9V to 8.9V.
Starting ramp from 8.9V to 8V with step 0.01V.
Successfully ramped from 8.9V to 7.99000000000002V.
Starting ramp from 8V to 7.9V with step 0.1V.
Successfully ramped from 8V to 7.9V.
Starting ramp from 7.9V to 7V with step 0.01V.
Successfully ramped from 7.9V to 6.99000000000002V.
Starting ramp from 7V to 6.9V with step 0.1V.
Successfully ramped from 7V to 6.9V.
Starting ramp from 6.9V to 6V with step 0.01V.
Successfully ramped from 6.9V to 5.99000000000002V.
Starting ramp from 6V to 5.9V with step 0.1V.
Successfully ramped from 6V to 5.9V.
Starting ramp from 5.9V to 5V with step 0.01V.
Successfully ramped from 5.9V to 4.99000000000002V.
Starting ramp from 5V to 4.9V with step 0.1V.
Successfully ramped from 5V to 4.9V.
Start

In [14]:

ramp_and_record(0,7,60,0.02)
record(120,"rec_at_7_part1")

ramp_and_record(7,10,60,0.01)
record(120,"rec_at_10")

ramp_and_record(10,7,60,0.01)
record(120,"rec_at_7_part2")

ramp_and_record(7,0,60,0.02)
record(120,"rec_at_0")


Starting ramp from 0V to 7V with step 0.02V.
current voltage: 0.02V.
current voltage: 0.04V.
current voltage: 0.06V.
current voltage: 0.08V.
current voltage: 0.1V.
current voltage: 0.12000000000000001V.
current voltage: 0.14V.
current voltage: 0.16V.
current voltage: 0.18V.
current voltage: 0.19999999999999998V.
current voltage: 0.21999999999999997V.
current voltage: 0.23999999999999996V.
current voltage: 0.25999999999999995V.
current voltage: 0.27999999999999997V.
current voltage: 0.3V.
current voltage: 0.32V.
current voltage: 0.34V.
current voltage: 0.36000000000000004V.
current voltage: 0.38000000000000006V.
current voltage: 0.4000000000000001V.
current voltage: 0.4200000000000001V.
current voltage: 0.4400000000000001V.
current voltage: 0.46000000000000013V.
current voltage: 0.48000000000000015V.
current voltage: 0.5000000000000001V.
current voltage: 0.5200000000000001V.
current voltage: 0.5400000000000001V.
current voltage: 0.5600000000000002V.
current voltage: 0.5800000000000002V.

Starting ramp from 7V to 10V with step 0.005V.
Successfully ramped from 7V to 10.000000000000291V.
Successfully recorded.


Starting ramp from 10V to 7V with step 0.005V.
Successfully ramped from 10V to 6.99999999999971V.
Successfully recorded.
Starting ramp from 7V to 0V with step 0.01V.
Successfully ramped from 7V to 1.0494036195574097e-13V.
Successfully recorded.


In [16]:
print(DAQ_data)

NameError: name 'DAQ_data' is not defined